In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from bs4 import BeautifulSoup
import io
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import sys
import re
from locale import atof, atoi, setlocale, LC_NUMERIC
import io
import json
import selenium.webdriver.support.select as select
from contextlib import contextmanager
from selenium.webdriver.support.expected_conditions import staleness_of
from selenium.webdriver.common.action_chains import ActionChains


In [2]:
setlocale(LC_NUMERIC, '')

'en_US.UTF-8'

In [3]:
# def wait_for(condition_function):
#   start_time = time.time()
#   while time.time() < start_time + 30:
#     if condition_function():
#       return True
#     else:
#       time.sleep(0.1)
#   raise Exception(
#    'Timeout waiting for {}'.format('shit')
#   )

In [4]:
# class wait_for_page_load(object):
#   def __init__(self, browser):
#     self.browser = browser
#   def __enter__(self):
#     self.old_page = self.browser.find_element_by_tag_name('html')
#   def page_has_loaded(self):
#     new_page = self.browser.find_element_by_tag_name('html')
#     return new_page.id != self.old_page.id
#   def __exit__(self, *_):
#     wait_for(self.page_has_loaded)

In [5]:
class scraper:
  def __init__(self, month='august', year=2020):
    self.paDriver = webdriver.chrome.webdriver.WebDriver('/usr/local/bin/chromedriver.85')
#     self.paDriver.get("https://www.thegrandwhiskyauction.com/past-auctions")
    self.month = month
    self.year = year
    self.urlBase = f"https://www.thegrandwhiskyauction.com/{month}-{year}"
#     self.urlBase = "https://www.thegrandwhiskyauction.com/september-auction"

#     self.paDriver.get(f"https://www.thegrandwhiskyauction.com/{self.month}-{self.year}")
    self.paDriver.get(f"{self.urlBase}/page-21")
    self.wait = WebDriverWait(self.paDriver, 10)
    try:
      elem = self.wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, \
                      "button[class='align-right secondary slidedown-button']")))    
      elem.click()
    except:
      pass
    print(self.paDriver.title)

#     assert "Past Auctions - Fine & Rare Scotch Whisky | The Grand Whisky Auction" in self.paDriver.title
    self.F = open(f'thegrandwhiskyauction.{self.month}-{self.year}.21.json', "wt")
    self.F.writelines('[\n')
#     self.html = BeautifulSoup(self.paDriver.page_source, 'html.parser')
    self.html = BeautifulSoup(self.paDriver.page_source, 'html.parser')      
    self.numberOfPages = self.getPages()
#     self.html = None
    self.ddriver =  webdriver.chrome.webdriver.WebDriver('/usr/local/bin/chromedriver.85')
    self.dwait = WebDriverWait(self.ddriver, 10)
    self.index = 0
    self.pageno = 21
#   @contextmanager
#   def wait_for_page_load(self, timeout=30):
#     old_page = self.paDriver.find_element_by_tag_name('html')
#     yield
#     WebDriverWait(self.paDriver, timeout).until(
#       staleness_of(old_page)
#     )    
    
  def wait_for_correct_current_url(self, desired_url):
      self.wait.until(
          lambda driver: driver.current_url == desired_url)
  def getPages(self):
    ul = self.html.find('ul', {"class": "pagination"})
    lis = ul.find_all('li')
    return atoi(str(lis[len(lis) - 2].getText()))
  def getMonthYear(self, month='september', year=2020):    
    elem = self.paDriver.find_element_by_css_selector(f"a[href='/{month}-{year}']")
    elem.click()
    self.html = BeautifulSoup(self.paDriver.page_source, 'html.parser')  
    self.month = month
    self.year = year
  def setPerPage(self):
    s = self.paDriver.find_element_by_css_selector("select[class='perPage fdropdown vueControlled']")
#     s.click()
    s = select.Select(s)
    dir(s)
    # s.select_by_index(4)
  def nextPage(self):
    if self.numberOfPages == self.pageno:
      return False
    tryNumber = 0
    while tryNumber < 3:
      try:
        ne = self.wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, \
                        "li[class='next']"))) 
      except:
        ne = None
  #     ne = self.paDriver.find_element_by_css_selector("li[class='next']")
      if ne is not None:
        self.paDriver.execute_script("arguments[0].scrollIntoView();", ne)
        ne.click()    
  #       actions = ActionChains(self.paDriver)
  #       actions.move_to_element(ne).perform()          
  #       ne.click()
        self.pageno += 1
#         newUrl = \
#           f"https://www.thegrandwhiskyauction.com/{self.month}-{self.year}/page-{self.pageno}"
        newUrl = f"{self.urlBase}/page-{self.pageno}"
        print('newUrl', newUrl)
        self.wait_for_correct_current_url(newUrl)
        self.html = BeautifulSoup(self.paDriver.page_source, 'html.parser') 
        return True
      if tryNumber < 2:
        ne = self.paDriver.find_element_by_css_selector("li[class='next']")      
        self.paDriver.execute_script("arguments[0].scrollIntoView();", ne)
        time.sleep(15)
      tryNumber += 1
    return False
  def close(self):
    offset = self.F.tell()
    print('offset', offset)
    self.F.seek(offset - 2)
    self.F.writelines('\n]\n')
    self.F.close()
  def dumpPage(self):
    ul = self.html.find('ul', {"class": "lotList"})
    lis = ul.find_all('li')
    print('len(lis)', len(lis))
    for li in lis: 
      nl = li.find('div', {"class": "nameAndLot"})
      a = nl.find('h3', {"itemprop": "name"}).find('a', href=True, title=True)
    #   print(a)
      name = str(a.string).strip()
      href = a["href"]
      ln = str(li.find('span', {"class": "lotNo"}).string).strip()[8:]
      wb = li.find('div', {"class": "bidPrice"}).find(text=re.compile('\s*Winning Bid\s*'))
      if wb is None:
        continue
      print('bidprice li name', name, 'usd', li.find('span', {"class": "USD show"}).string)
      price = atof(str(li.find('span', {"class": "USD show"}).string)[3:])
      print('href for ddriver', href)
      tryNumber = 0
      while tryNumber < 3:
        self.ddriver.execute_script(f"window.location.href='{href}'")
  #       self.ddriver.get(href)
        if self.index == 0:
          try:
            elem = self.dwait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, \
                        "button[class='align-right secondary slidedown-button']")))    
            elem.click()
          except:
            pass
        self.index += 1
        try:
          self.dwait.until(EC.presence_of_element_located((By.CSS_SELECTOR, \
                            "div[class='descriptionWrap']")))
        except:
          tryNumber += 1
          if tryNumber == 3:
            print("timeouterror can't read", href)
            raise TimeoutError
          time.sleep(5)
          continue
        break
      dsource = self.ddriver.page_source
      dhtml = BeautifulSoup(dsource, 'html.parser')  
      dw = dhtml.find('div', {'class': "descriptionWrap"})
      dwt = dw.find('div', {'class': "innerText pageContent"})
      dps = dwt.find_all("p")
      description = ""
      for p in dps:
#         print('p', p)
        description += str(p.getText()) + '\n'
        
      dul = dhtml.find('ul', {"class": "lotProps"})
      dlis = dul.find_all('li')
      distillery_status = strength = type_ = size = region = country = bottler = image_url = \
            number_of_bottles_produced = age = distillery = None
#       distillery = name
      
      for dli in dlis:
        ts = dli.find_all(text=re.compile("[\w\s]+"))
        
        j = 0
        if len(ts) > 0:
          ts0 = ts[0]
          if len(ts0.strip()) == 0:
            j += 1
        else:
          continue
        if len(ts) > j:
          ts0 = str(ts[j]).strip()
        else:
          continue
        if len(ts) > j + 1:
          ts1 = str(ts[j + 1]).strip()
          if ts1 == 'NA' or ts1 == 'N/A' or len(ts1) == 0:
            ts1 = None
        else:
          ts1 = None
#         print(f"ts0 {ts0}  ts1 {ts1}")
        if ts0 == "Distillery:":
          dis0 = dli.find('img', src=True, alt=True)
          if dis0 is not None:
            image_url = dis0['src']
            distillery = dis0['alt']
          else:
            distillery = ts1
          continue
        else:
          if ts1 is None:
            continue
        if ts0 == "Bottler:":
          bottler = ts1
        elif ts0 == "Country:":
          country = ts1
        elif ts0 == "Region:":
          region = ts1
        elif ts0 == "Size:":
          size = ts1
        elif ts0 == "Type:":
          type_ = ts1
        elif ts0 == "Strength (%):":
          strength = ts1
        elif ts0 == "Distillery Status:":
          distillery_status = ts1
        elif ts0 == "Age:":
          try: 
            age = atoi(ts1[:2])
          except:
            age = ts1
        elif ts0 == "Number of Bottles:":
          try:
            number_of_bottles_produced = atof(ts1)
          except:
            number_of_bottles_produced = ts1
          
      json.dump({"NAME": name, "WINNING BID": price, "DISTILLERY / BLENDER": distillery, "LOT #": ln, \
             "BOTTLER": bottler, "COUNTRY": country, "REGION": region, "TYPE": type_, "AGE": age, \
             "BOTTLES PRODUCED": number_of_bottles_produced, "STRENGTH": strength, \
             "IMAGE URL": image_url, "SOURCE URL": href, "DISTILLERY STATUS": distillery_status, \
             "SIZE": size, "MONTH": f"{self.month} {self.year}", "DESCRIPTION": description
                }, self.F)
      self.F.writelines(',\n')
            

In [12]:
s = scraper()

Browse Whisky Auctions | The Grand Whisky Auction


In [30]:
atof("77,748.00")

77748.0

In [39]:
s.getMonthYear()

In [48]:
s.html = BeautifulSoup(s.paDriver.page_source, 'html.parser') 

In [73]:
s.setPerPage()

In [78]:
ul = s.html.find('ul', {"class": "lotList"})

In [79]:
type(ul)

bs4.element.Tag

In [13]:
s.dumpPage()

len(lis) 36
bidprice li name Bowmore - Devil's Cask - Batch III - Double the Devil usd US$194.37
href for ddriver https://www.thegrandwhiskyauction.com/lot-161195/bowmore-devils-cask-batch-iii-double-the-devil/auction-20
bidprice li name Dalmore - Distillery Exclusive 2019 Cabernet Sauvignon Finish 2007 usd US$194.37
href for ddriver https://www.thegrandwhiskyauction.com/lot-160440/dalmore-distillery-exclusive-2019-cabernet-sauvignon-finish-2007/auction-20
bidprice li name Bimber - Oloroso Cask - Small Batch #1 usd US$194.37
href for ddriver https://www.thegrandwhiskyauction.com/lot-161211/bimber-oloroso-cask-small-batch-1/auction-20
bidprice li name Caperdonich - 21 Year Old Peated - Distillery Reserve 1996 usd US$194.37
href for ddriver https://www.thegrandwhiskyauction.com/lot-160455/caperdonich-21-year-old-peated-distillery-reserve-1996/auction-20
bidprice li name Ardbeg - 10 Year Old Still Young 1998-2008 2nd Release usd US$194.37
href for ddriver https://www.thegrandwhiskyauction

In [8]:
# s.urlBase = "https://www.thegrandwhiskyauction.com/december-2019"

In [ ]:
ne = s.wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, \
                "li[class='next']")))
actions = ActionChains(s.paDriver)
actions.move_to_element(ne)
ne.click()



In [ ]:
ne = s.wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, \
                "li[class='next']")))
s.paDriver.execute_script("arguments[0].scrollIntoView();", ne)
ne.click()

In [ ]:
dsource = s.ddriver.page_source
dhtml = BeautifulSoup(dsource, 'html.parser')  
dw = dhtml.find('div', {'class': "descriptionWrap"})


In [ ]:
dwt = dw.find('div', {'class': "innerText pageContent"})
dps = dwt.find_all("p")
len(dps)

In [14]:
while s.nextPage():
  s.dumpPage()

newUrl https://www.thegrandwhiskyauction.com/august-2020/page-22
len(lis) 36
bidprice li name Laphroiag - 18 Year Old 2004 - Old Particular - Queen of the Hebrides usd US$181.41
href for ddriver https://www.thegrandwhiskyauction.com/lot-161597/laphroiag-18-year-old-2004-old-particular-queen-of-the-hebrides/auction-20
bidprice li name Macallan - Rare Cask usd US$181.41
href for ddriver https://www.thegrandwhiskyauction.com/lot-161352/macallan-rare-cask/auction-20
bidprice li name Bruichladdich - 1986 Valinch - Continuation of the Celebration usd US$181.41
href for ddriver https://www.thegrandwhiskyauction.com/lot-161106/bruichladdich-1986-valinch-continuation-of-the-celebration/auction-20
bidprice li name Glenfiddich - Winter Storm #3 usd US$181.41
href for ddriver https://www.thegrandwhiskyauction.com/lot-161360/glenfiddich-winter-storm-3/auction-20
bidprice li name Caol Ila - 16 Year Old - Fèis Ìle 2020 usd US$181.41
href for ddriver https://www.thegrandwhiskyauction.com/lot-161114/ca

bidprice li name Waterford - Single Farm Origin - Ballykilcavan 1.2 usd US$168.45
href for ddriver https://www.thegrandwhiskyauction.com/lot-161824/waterford-single-farm-origin-ballykilcavan-12/auction-20
bidprice li name Talisker - 8 Year Old 2009 Cask Strength Limited Edition usd US$168.45
href for ddriver https://www.thegrandwhiskyauction.com/lot-161318/talisker-8-year-old-2009-cask-strength-limited-edition/auction-20
bidprice li name Arran - 20 Year Old - The Explorers Series - Brodick Bay - Vol.1 usd US$168.45
href for ddriver https://www.thegrandwhiskyauction.com/lot-161322/arran-20-year-old-the-explorers-series-brodick-bay-vol1/auction-20
bidprice li name Tamdhu - 10 Year Old  - 1980s usd US$168.45
href for ddriver https://www.thegrandwhiskyauction.com/lot-161841/tamdhu-10-year-old-1980s/auction-20
bidprice li name Highland Park - 13 Year Old - Single Cask Series Germany #6353 usd US$168.45
href for ddriver https://www.thegrandwhiskyauction.com/lot-160079/highland-park-13-year-o

bidprice li name Laphroaig - Cairdeas - Master Edition usd US$168.45
href for ddriver https://www.thegrandwhiskyauction.com/lot-160751/laphroaig-cairdeas-master-edition/auction-20
bidprice li name Laphroaig - 10 Year Old - Original Cask Strength Batch #007 usd US$168.45
href for ddriver https://www.thegrandwhiskyauction.com/lot-160753/laphroaig-10-year-old-original-cask-strength-batch-007/auction-20
bidprice li name Bunnahabhain - 2003 - Amontillado Cask Finish usd US$168.45
href for ddriver https://www.thegrandwhiskyauction.com/lot-160755/bunnahabhain-2003-amontillado-cask-finish/auction-20
bidprice li name Bellevue - 19 Year Old  1998 The Duchess Single Cask Guadeloupe Rum usd US$168.45
href for ddriver https://www.thegrandwhiskyauction.com/lot-161510/bellevue-19-year-old-1998-the-duchess-single-cask-guadeloupe-rum/auction-20
bidprice li name Laphroaig - Cairdeas - Master Edition usd US$168.45
href for ddriver https://www.thegrandwhiskyauction.com/lot-160761/laphroaig-cairdeas-master

len(lis) 36
bidprice li name Clynelish - 14 Year Old - Flora and Fauna usd US$155.50
href for ddriver https://www.thegrandwhiskyauction.com/lot-161924/clynelish-14-year-old-flora-and-fauna/auction-20
bidprice li name Dalwhinnie - 12 Year Old - Manager's Dram 2009 usd US$155.50
href for ddriver https://www.thegrandwhiskyauction.com/lot-160671/dalwhinnie-12-year-old-managers-dram-2009/auction-20
bidprice li name Macallan - 12 Year Old - Fine Oak - Year of the Rat (2 x 70cl) usd US$155.50
href for ddriver https://www.thegrandwhiskyauction.com/lot-161424/macallan-12-year-old-fine-oak-year-of-the-rat-2-x-70cl/auction-20
bidprice li name Glenrothes - 20 Year Old 1989 First Cask usd US$155.50
href for ddriver https://www.thegrandwhiskyauction.com/lot-161179/glenrothes-20-year-old-1989-first-cask/auction-20
bidprice li name Highland Park - 18 Year Old 1989 First Cask usd US$155.50
href for ddriver https://www.thegrandwhiskyauction.com/lot-161180/highland-park-18-year-old-1989-first-cask/auctio

bidprice li name Balvenie - 15 Year Old Single Barrel #11168 usd US$142.54
href for ddriver https://www.thegrandwhiskyauction.com/lot-161077/balvenie-15-year-old-single-barrel-11168/auction-20
bidprice li name Macallan - Edition No.4 usd US$142.54
href for ddriver https://www.thegrandwhiskyauction.com/lot-160317/macallan-edition-no4/auction-20
bidprice li name Lagavulin - 10 Year Old - Travel Exclusive usd US$142.54
href for ddriver https://www.thegrandwhiskyauction.com/lot-160833/lagavulin-10-year-old-travel-exclusive/auction-20
bidprice li name Highland Park - 13 Year Old - Single Cask #2865 - Draken (75cl) usd US$142.54
href for ddriver https://www.thegrandwhiskyauction.com/lot-160074/highland-park-13-year-old-single-cask-2865-draken-75cl/auction-20
bidprice li name Highland Park - 18 Year Old usd US$142.54
href for ddriver https://www.thegrandwhiskyauction.com/lot-161335/highland-park-18-year-old/auction-20
bidprice li name Bowmore - 15 Year Old - Distillery Excluisve - Fèis Ìle 20

...

bidprice li name Longrow - Peated usd US$51.83
href for ddriver https://www.thegrandwhiskyauction.com/lot-161789/longrow-peated/auction-20
bidprice li name Tomatin - 12 Year Old French Oak 75cl usd US$51.83
href for ddriver https://www.thegrandwhiskyauction.com/lot-160785/tomatin-12-year-old-french-oak-75cl/auction-20
bidprice li name Glenfarclas - Private Reserve Oak Casks usd US$51.83
href for ddriver https://www.thegrandwhiskyauction.com/lot-161052/glenfarclas-private-reserve-oak-casks/auction-20
bidprice li name Glenfarclas - Private Reserve Oak Casks usd US$51.83
href for ddriver https://www.thegrandwhiskyauction.com/lot-161053/glenfarclas-private-reserve-oak-casks/auction-20
bidprice li name Bruichladdich - 10 Year Old usd US$51.83
href for ddriver https://www.thegrandwhiskyauction.com/lot-161054/bruichladdich-10-year-old/auction-20
bidprice li name Haig's - Dimple '1890' 50cl usd US$51.35
href for ddriver https://www.thegrandwhiskyauction.com/lot-160428/haigs-dimple-1890-50cl/au

bidprice li name Royal Brackla - 10 Year Old Navy Wings 2006 usd US$45.35
href for ddriver https://www.thegrandwhiskyauction.com/lot-160790/royal-brackla-10-year-old-navy-wings-2006/auction-20
bidprice li name Orkney - 9 Year Old - Creative Whisky Co. Single Cask usd US$45.35
href for ddriver https://www.thegrandwhiskyauction.com/lot-160796/orkney-9-year-old-creative-whisky-co-single-cask/auction-20
bidprice li name Glenfarclas - Private Reserve Oak Casks usd US$45.35
href for ddriver https://www.thegrandwhiskyauction.com/lot-161051/glenfarclas-private-reserve-oak-casks/auction-20
bidprice li name Deanston - Dragon’s Milk Stout Cask Finish usd US$45.35
href for ddriver https://www.thegrandwhiskyauction.com/lot-161811/deanston-dragons-milk-stout-cask-finish/auction-20
bidprice li name Teeling  - Stout Cask Finish usd US$45.35
href for ddriver https://www.thegrandwhiskyauction.com/lot-161559/teeling-stout-cask-finish/auction-20
newUrl https://www.thegrandwhiskyauction.com/august-2020/pag

bidprice li name An Cnoc - 12 Years Old usd US$32.40
href for ddriver https://www.thegrandwhiskyauction.com/lot-161890/an-cnoc-12-years-old/auction-20
bidprice li name Glen Moray - 12 Year Old (1980s) usd US$32.40
href for ddriver https://www.thegrandwhiskyauction.com/lot-161642/glen-moray-12-year-old-1980s/auction-20
bidprice li name Ice & Fire - Caithness Raiders Rum usd US$32.40
href for ddriver https://www.thegrandwhiskyauction.com/lot-161902/ice-fire-caithness-raiders-rum/auction-20
bidprice li name Bowmore - No.1 usd US$32.40
href for ddriver https://www.thegrandwhiskyauction.com/lot-160904/bowmore-no1/auction-20
bidprice li name Royal Lochnagar - 12 Year Old usd US$32.40
href for ddriver https://www.thegrandwhiskyauction.com/lot-160395/royal-lochnagar-12-year-old/auction-20
bidprice li name House of Bruar - Highland Single Malt usd US$32.40
href for ddriver https://www.thegrandwhiskyauction.com/lot-161168/house-of-bruar-highland-single-malt/auction-20
bidprice li name Spirit Of 

bidprice li name Glen Dochart - 8 Year Old - Pure Malt (1980s) usd US$25.92
href for ddriver https://www.thegrandwhiskyauction.com/lot-161863/glen-dochart-8-year-old-pure-malt-1980s/auction-20
bidprice li name Glen Darbach - 12 Year Old usd US$25.92
href for ddriver https://www.thegrandwhiskyauction.com/lot-161867/glen-darbach-12-year-old/auction-20
bidprice li name Grant's - Family Reserve usd US$25.92
href for ddriver https://www.thegrandwhiskyauction.com/lot-160153/grants-family-reserve/auction-20
bidprice li name Jura - Journey usd US$25.92
href for ddriver https://www.thegrandwhiskyauction.com/lot-161674/jura-journey/auction-20
bidprice li name Tamnavulin - Double Cask usd US$25.92
href for ddriver https://www.thegrandwhiskyauction.com/lot-160160/tamnavulin-double-cask/auction-20
bidprice li name Inverey - 12 Year Old Highland Single Malt usd US$25.92
href for ddriver https://www.thegrandwhiskyauction.com/lot-161174/inverey-12-year-old-highland-single-malt/auction-20
bidprice li n

bidprice li name Famous Grouse - Mini Set (2 x 5cl) usd US$12.96
href for ddriver https://www.thegrandwhiskyauction.com/lot-160159/famous-grouse-mini-set-2-x-5cl/auction-20
newUrl https://www.thegrandwhiskyauction.com/august-2020/page-52
len(lis) 18
bidprice li name Assorted Minis - (10 x 5cl) usd US$12.96
href for ddriver https://www.thegrandwhiskyauction.com/lot-160191/assorted-minis-10-x-5cl/auction-20
bidprice li name Assorted Minis - (10 x 5cl) usd US$12.96
href for ddriver https://www.thegrandwhiskyauction.com/lot-160192/assorted-minis-10-x-5cl/auction-20
bidprice li name Assorted Minis - (10 x 5cl) usd US$11.66
href for ddriver https://www.thegrandwhiskyauction.com/lot-160193/assorted-minis-10-x-5cl/auction-20
bidprice li name Gordon & MacPhail - Minis (3 x 5cl) usd US$10.37
href for ddriver https://www.thegrandwhiskyauction.com/lot-160161/gordon-macphail-minis-3-x-5cl/auction-20
bidprice li name Assorted Minis - (x3) usd US$10.37
href for ddriver https://www.thegrandwhiskyaucti

In [ ]:
s.F.writelines(']\n')

In [178]:
se = s.paDriver.find_element_by_css_selector("select[class='perPage fdropdown vueControlled']")

In [15]:
s.close()

offset 892003


In [160]:
se.click()

NameError: name 'se' is not defined

In [50]:
sel = select.Select(se)

In [55]:
sel.select_by_index(3)

ElementNotInteractableException: Message: element not interactable: Element is not currently visible and may not be manipulated
  (Session info: chrome=85.0.4183.102)


In [56]:
se = s.paDriver.find_element_by_css_selector("a[rel='prev']")

In [57]:
se.click()

In [5]:
driver = webdriver.chrome.webdriver.WebDriver('/usr/local/bin/chromedriver.85')
driver.get("https://www.thegrandwhiskyauction.com/past-auctions")
print(driver.title)
assert "Past Auctions - Fine & Rare Scotch Whisky | The Grand Whisky Auction" in driver.title
urls = []

Past Auctions - Fine & Rare Scotch Whisky | The Grand Whisky Auction


In [90]:
paDriver = webdriver.chrome.webdriver.WebDriver('/usr/local/bin/chromedriver.85')

In [91]:
paDriver.get("https://www.thegrandwhiskyauction.com/past-auctions")

In [99]:
elem = paDriver.find_element_by_css_selector("button[class='align-right secondary slidedown-button']")

In [100]:
elem.click()

In [92]:
html = BeautifulSoup(paDriver.page_source, 'html.parser')  

In [125]:
type(s.F)

_io.TextIOWrapper

In [94]:
elem = html.find('button', {"class": "align-right secondary slidedown-button"})

In [97]:
elem

<button class="align-right secondary slidedown-button" id="onesignal-slidedown-cancel-button">No Thanks :-(</button>

In [98]:
elem.click()

TypeError: 'NoneType' object is not callable

In [11]:
elem = driver.find_element_by_css_selector("a[href='/september-2020']")
elem.click()

In [13]:
source = driver.page_source
html = BeautifulSoup(source, 'html.parser')

In [16]:
ul = html.find('ul', {"class": "lotList"})

In [21]:
type(ul)

bs4.element.Tag

In [22]:
lis = ul.find_all('li')

In [101]:
with open('thegrandwhiskyauction.json', "wt") as F:
  for li in lis:
    nl = li.find('div', {"class": "nameAndLot"})
    a = nl.find('h3', {"itemprop": "name"}).find('a', href=True, title=True)
  #   print(a)
    name = str(a.string).strip()
    href = a["href"]
    ln = str(li.find('span', {"class": "lotNo"}).string).strip()[8:]
    wb = li.find('div', {"class": "bidPrice"}).find(text=re.compile('\s*Winning Bid\s*'))
    if wb is None:
      continue
    price = atof(str(li.find('span', {"class": "USD show"}).string)[3:])
    driver.get(href)
    dsource = driver.page_source
    dhtml = BeautifulSoup(dsource, 'html.parser')  
    dul = dhtml.find('ul', {"class": "lotProps"})
    dlis = dul.find_all('li')
    for dli in dlis
    
    dis0 = dlis[0].find('img', src=True, alt=True)
    image_url = dis0['src']
    distillery = dis0['alt']
    bottler = str(dlis[1].find_all(text=re.compile("\s*\w+\s*"))[1]).strip()
    country = str(dlis[2].find_all(text=re.compile("\s*\w+\s*"))[1]).strip()
    region = str(dlis[3].find_all(text=re.compile("\s*\w+\s*"))[1]).strip()
    size = str(dlis[4].find_all(text=re.compile("\s*\w+\s*"))[1]).strip()
    type_ = str(dlis[5].find_all(text=re.compile("\s*\w+\s*"))[1]).strip()    
    age = atoi(str(dlis[6].find_all(text=re.compile("\s*\w+\s*"))[1]).strip()[:2])
    number_of_bottles_produced = atoi(str(dlis[7].find_all(text=re.compile("\s*\w+\s*"))[1]).strip())
    strength = str(dlis[8].find_all(text=re.compile("\s*\w+\s*"))[1]).strip()
    

    json.dump({"name": name, "href": href , "lotNo": ln, "price": price, "distillery": distillery, \
             "bottler": bottler, "country": country, "region": region, "type": type_, "age": age, \
             "number_of_bottles_produced": number_of_bottles_produced, "strength": strength, \
             "image_url": image_url, "source_url": href }, F)
#     break

  
  

TypeError: 'NoneType' object is not subscriptable

In [96]:
r

[{'name': 'Macallan - 72 Year Old Lalique Genesis Decanter',
  'href': 'https://www.thegrandwhiskyauction.com/lot-170490/macallan-72-year-old-lalique-genesis-decanter/auction-21',
  'lotNo': '170490',
  'price': 77292.0,
  'distillery': 'Macallan Whisky for auction',
  'bottler': 'Distillery'}]